# Imports

In [ ]:
import json
import os
import pickle
import warnings
from ast import literal_eval

import awkward as ak
import latexify
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as signal
import uproot
from itables import init_notebook_mode
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.ticker import AutoMinorLocator, MaxNLocator
from scipy import stats
from scipy.optimize import curve_fit
from skimage.measure import LineModelND, ransac
from sklearn.cluster import DBSCAN, KMeans
from sklearn.linear_model import RANSACRegressor
from skspatial.objects import Cylinder, Line, Point, Triangle
from tqdm.auto import tqdm

init_notebook_mode(all_interactive=True)

# Parameters

In [ ]:
# File paths
file_label = "20230706_191437"  # label for files generated by this notebook
light_file = "../Data/rwf_0cd913fa_20230706_191437.data_deco_2.root"
charge_file = "../Data/evd_self_trigger-packets-2023_07_06_19_14_CEST_validated.root"
sipm_map_file = "sipm_sn_ch_to_xy.json"

# Load options
reload_file = False
rematch_events = False

# Plotting options
individual_plots = []  # np.arange(1, 21, 1)
show_figures = True

# Events to process
event_list = None
# event_list = np.array([30, 50])
# event_list = np.arange(0, 20, 1)

# Noisy Pixels
channel_disable_list = [7]

# Light variable to consider
light_variable = "integral"

# Units for plot labels
q_unit = "ke"  # After applying charge_gain
xy_unit = "mm"
z_unit = "mm"
dh_unit = "?" if z_unit != xy_unit else xy_unit
time_unit = "ns"
light_unit = "p.e." if light_variable == "integral" else "p.e./time bin"

# Conversion factors
charge_gain = 245  # mV to ke
detector_z = 300
detector_x = 128
detector_y = 160

# DBSCAN parameters for clustering
min_samples = 2
xy_epsilon = 8  # 8 ideal
z_epsilon = 8  # 8 ideal

# RANSAC parameters for line fitting
residual_threshold = 6  # 6 ideal
max_trials = 1000

# Force parameters for cylinder
force_dh = None
force_dr = None

# Filters for post processing
p_value_limit = 0.05
min_track_length = 160
max_tracks = 1

# Functions

## Parameter calculators

In [ ]:
# SiPMs mapping
sipm_map = None


def sipm_to_xy(sn, ch):
    global sipm_map
    if sipm_map is None:
        with open(sipm_map_file, "r") as f:
            sipm_map = json.load(f)

    xy = sipm_map.get(str(sn), {}).get(str(ch), None)
    if xy is None:
        return None
    else:
        x = xy[0] + 64
        y = xy[1] - 16
        return (x, y)


# Check if SiPMs on anode area
def get_sipm_mask(sn, ch):
    xy = sipm_to_xy(sn, ch)
    # return True
    if xy is None:
        return False
    else:
        return (
            xy[0] > -detector_x / 2
            and xy[0] < detector_x / 2
            and xy[1] < detector_y / 2
            and xy[1] > -detector_y / 2
        )

In [ ]:
# Cylinder parameters for dQ/dx
def get_dh(unit_vector):
    if force_dh is not None:
        return force_dh

    dl_vector = np.array([xy_epsilon, xy_epsilon, z_epsilon])
    min_dh = np.linalg.norm(dl_vector) / 4
    max_dh = 2 * np.linalg.norm(dl_vector)
    dl_projection = abs(np.dot(unit_vector, dl_vector))
    dh = min(max(dl_projection, min_dh), max_dh)

    return dh


def get_dr(standard_deviation):
    if force_dr is not None:
        return force_dr

    dl_vector = np.array([xy_epsilon, xy_epsilon, z_epsilon])
    min_dr = np.linalg.norm(dl_vector) / 4
    dr = max(standard_deviation, min_dr)

    return dr

## Uproot functions

In [ ]:
# Uproot
def load_charge(file_name, events=None):
    with uproot.open(file_name) as f:
        charge_df = f["events"].arrays(library="pd").set_index("eventID")
        if events is not None:
            charge_df = charge_df.loc[events]

    return charge_df


def load_light(file_name, deco=True, events=None, mask=True, keep_rwf=False):
    light_df = pd.DataFrame()
    with uproot.open(file_name) as f:
        if deco:
            tree = f["decowave"]
        else:
            tree = f["rwf_array"]

        for idx, arrays in enumerate(tree.iterate(library="np")):
            df = pd.DataFrame.from_dict(arrays, orient="index").T
            df.dropna()
            if events is not None:
                df = df[df["event"].isin(events)]

            if mask:
                df = df[
                    df[["sn", "ch"]].apply(lambda x: get_sipm_mask(x[0], x[1]), axis=1)
                ]

            if df.empty:
                continue

            df[["x", "y"]] = df[["sn", "ch"]].apply(
                lambda x: pd.Series(sipm_to_xy(x[0], x[1])), axis=1
            )

            if deco:
                df["rwf"] = df["decwfm"]

            df[["integral", "properties"]] = df["rwf"].apply(
                lambda x: pd.Series(
                    (np.nan, {}) if any(np.isnan(x)) else integrate_peaks(x)
                )
            )
            df["peak"] = df["properties"].apply(
                lambda x: max(x["peak_heights"])
                if "peak_heights" in x and len(x["peak_heights"]) > 0
                else np.nan
            )

            columns = ["event", "tai_ns", "sn", "ch", "peak", "integral", "x", "y"]
            if keep_rwf:
                columns.append("rwf")

            df = df[columns]
            light_df = pd.concat([light_df, df], ignore_index=True)

    return light_df

## Data handling

In [ ]:
def filter_metrics(metrics, p_value_limit=0.05, min_track_length=160, max_tracks=1):
    filtered_metrics = {}
    non_track_keys = 4

    for event_idx, metric in metrics.items():
        if len(metric) == max_tracks + non_track_keys:
            candidate_metric = {
                track_idx: values
                for track_idx, values in metric.items()
                if isinstance(track_idx, str)
                or (
                    track_idx > 0
                    and values["Fit_p_value"] <= p_value_limit
                    and values["Fit_norm"] >= min_track_length
                )
            }
            if len(candidate_metric) == max_tracks + non_track_keys:
                filtered_metrics[event_idx] = candidate_metric

    return filtered_metrics

In [ ]:
# Peak finding algorithm for integration
def integrate_peaks(waveform, buffer_size=10, height=0.1, prominence=0.05):
    # Find peaks in the filtered waveform
    peaks, properties = signal.find_peaks(
        waveform, height=height, prominence=prominence
    )

    integration_result = 0
    start_index = 0  # Initialize the start index
    end_index = 0  # Initialize the end index

    for peak in peaks:
        # Determine the potential start and end indices
        potential_start = max(0, peak - buffer_size)
        potential_end = min(len(waveform), peak + buffer_size)

        # If the potential start is within the current peak region, update the end index
        if potential_start <= end_index:
            end_index = potential_end
        else:
            # Integrate the previous peak region and update indices for the new peak
            peak_region = waveform[start_index:end_index]
            integration_result += np.trapz(peak_region)
            start_index = potential_start
            end_index = potential_end

    # Integrate the last peak region
    peak_region = waveform[start_index:end_index]
    integration_result += np.trapz(peak_region)

    return integration_result, properties

In [ ]:
match_dict = {}


def match_events(charge_df, light_df, window=10):
    match_dict = {}

    charge_events = charge_df[["event_unix_ts", "event_start_t"]].drop_duplicates()
    light_events = light_df[["tai_ns", "event"]].drop_duplicates()

    for event, row in tqdm(charge_events.iterrows(), total=len(charge_events)):
        charge_ts = (float(row["event_unix_ts"]) * 1e6) + (
            float(row["event_start_t"]) * 1e-1
        )
        light_matches = light_events.where(
            abs(light_events["tai_ns"].astype(float) * 1e-3 - 36000000 - charge_ts)
            <= window
        ).dropna()

        if not light_matches.empty:
            if event in match_dict:
                match_dict[event].append(
                    light_matches["event"].unique().astype(int).tolist()
                )
            else:
                match_dict[event] = light_matches["event"].unique().astype(int).tolist()

    return match_dict

In [ ]:
# Create a list of fake data
def generate_dead_area(z_range):
    # Dead area on chips 44, 54, detector_x/2
    fake_x1, fake_y1, fake_z1 = np.meshgrid(
        np.linspace(36, 60, 6),
        np.concatenate(
            [
                np.linspace(-76, -52, 6),
                np.linspace(-44, -20, 6),
                np.linspace(-12, 12, 6),
            ]
        ),
        z_range,
    )

    # Dead area on chip 33
    fake_x2, fake_y2, fake_z2 = np.meshgrid(
        np.linspace(4, 28, 6), np.linspace(20, 44, 6), z_range
    )

    # Dead area on chip 42
    fake_x3, fake_y3, fake_z3 = np.meshgrid(
        np.linspace(-28, -4, 6), np.linspace(-12, 12, 6), z_range
    )
    mask = fake_x3 + (fake_y3 + 16) <= 0
    fake_x3, fake_y3, fake_z3 = fake_x3[mask], fake_y3[mask], fake_z3[mask]

    fake_x4, fake_y4, fake_z4 = np.meshgrid([-14], [2], z_range)

    # Dead area on SiPMs
    fake_x5 = []
    fake_y5 = []
    fake_z5 = []
    for k in range(4):
        for l in range(5):
            if (k == 3 and l < 3) or (k == 2 and l == 3) or (k == 1 and l == 2):
                continue

            temp_x, temp_y, temp_z = np.meshgrid(
                np.array([-50, -46]) + 32 * k,
                np.array([-66, -62]) + 32 * l,
                z_range,
            )

            fake_x5.extend(temp_x)
            fake_y5.extend(temp_y)
            fake_z5.extend(temp_z)

    fake_x5 = np.array(fake_x5)
    fake_y5 = np.array(fake_y5)
    fake_z5 = np.array(fake_z5)

    # Concatenate all the fake data
    fake_x = np.concatenate(
        [
            fake_x1.flatten(),
            fake_x2.flatten(),
            fake_x3.flatten(),
            fake_x4.flatten(),
            fake_x5.flatten(),
        ]
    )
    fake_y = np.concatenate(
        [
            fake_y1.flatten(),
            fake_y2.flatten(),
            fake_y3.flatten(),
            fake_y4.flatten(),
            fake_y5.flatten(),
        ]
    )
    fake_z = np.concatenate(
        [
            fake_z1.flatten(),
            fake_z2.flatten(),
            fake_z3.flatten(),
            fake_z4.flatten(),
            fake_z5.flatten(),
        ]
    )

    fake_data = np.c_[fake_x, fake_y, fake_z]

    return fake_data

In [ ]:
# Apply DBSCAN clustering
def cluster(hitArray):
    # First stage clustering
    z_intervals = []
    first_stage = DBSCAN(eps=xy_epsilon, min_samples=min_samples).fit(hitArray[:, 0:2])
    for label in first_stage.labels_:
        if label > -1:
            mask = first_stage.labels_ == label
            z = hitArray[mask, 2]
            z_intervals.append((min(z), max(z)))

    # Sort the intervals based on their start points
    sorted_intervals = sorted(z_intervals, key=lambda interval: interval[0])

    # Initialize a list to store the intervals representing the empty space
    empty_space_ranges = []

    # Iterate through the sorted intervals to find the gaps
    for i in range(len(sorted_intervals) - 1):
        current_interval = sorted_intervals[i]
        next_interval = sorted_intervals[i + 1]

        # Calculate the gap between the current interval and the next interval
        gap_start = current_interval[1]
        gap_end = next_interval[0]

        # Check if there is a gap (empty space) between intervals
        if gap_end > gap_start and gap_end < gap_start + 40:
            empty_space_ranges.append(np.arange(gap_start, gap_end, z_epsilon))

    if not empty_space_ranges:
        empty_space_ranges.append(
            np.arange(
                np.mean(hitArray[:, 2]) - np.std(hitArray[:, 2]),
                np.mean(hitArray[:, 2]) + np.std(hitArray[:, 2]),
                z_epsilon,
            )
        )

    z_range = np.concatenate(empty_space_ranges)

    # Create a list of holes
    fake_data = generate_dead_area(z_range)
    fake_data_count = len(fake_data)

    # Second stage clustering
    # Combine fake to true data
    second_stage_data = np.concatenate([hitArray, fake_data])
    second_stage = DBSCAN(eps=xy_epsilon, min_samples=1).fit(second_stage_data[:, 0:2])

    # Third stage clustering
    # Create a new array with z and labels
    third_stage_z = np.c_[second_stage.labels_ * 1e3, second_stage_data[:, 2]]
    flag = second_stage.labels_ > -1
    third_stage_data = third_stage_z[flag].copy()
    third_stage = DBSCAN(
        eps=z_epsilon, min_samples=min_samples, metric="chebyshev"
    ).fit(third_stage_data)

    # Remove fake data
    # Shift labels by 1 so that negative values are reserved for outliers
    labels = third_stage.labels_[:-fake_data_count] + 1

    return labels


# Apply Ransac Fit
def ransacFit(hitArray, weightArray=None):
    if weightArray is not None:
        estimator = RANSACRegressor(
            min_samples=min_samples,
            max_trials=max_trials,
            residual_threshold=residual_threshold,
        )
        inliers = estimator.fit(
            hitArray[:, 0:2],
            hitArray[:, 2],
            sample_weight=weightArray,
        ).inlier_mask_

        score = estimator.score(hitArray[:, 0:2], hitArray[:, 2])
    else:
        model_robust, inliers = ransac(
            hitArray,
            LineModelND,
            min_samples=min_samples,
            residual_threshold=residual_threshold,
            max_trials=max_trials,
        )

        score = model_robust.score(hitArray)

    outliers = inliers == False
    return inliers, outliers, score


# Apply best line fit
def lineFit(hitArray):
    line = Line.best_fit(hitArray)
    residuals = []
    for point in hitArray:
        distance = line.distance_point(point)
        residuals.append(distance)

    # Convert residuals to a numpy array
    residuals = np.array(residuals)

    # Calculate chi-squared
    chi_squared = np.sum(residuals**2)

    return line, chi_squared


# Calculate dQ/dx from a line fit
def dqdx(hitArray, q, line_fit, dh, dr, h, ax=None):
    # Cylinder steps for dQ/dx
    steps = np.arange(-3 * dh, h + 2 * dh, dh)

    # Mask of points that have been accounted for
    counted = np.zeros(len(q), dtype=bool)

    # Array of dQ values for each step
    dq_i = np.zeros(len(steps), dtype=float)

    for step_idx, step in enumerate(steps):
        cylinder_fit = Cylinder(
            line_fit.to_point(h / 2 - step),
            -line_fit.direction.unit() * dh,
            dr,
        )
        if ax is not None:
            cylinder_fit.plot_3d(ax)

        for point_idx, point in enumerate(hitArray):
            if not counted[point_idx] and cylinder_fit.is_point_within(point):
                counted[point_idx] = True
                dq_i[step_idx] += q[point_idx]

    return dq_i


# Fit clusters with Ransac method
def fitClusters(
    hitArray, q, labels, ax2d=None, ax3d=None, plot_cyl=False, refit_outliers=True
):
    metrics = {}
    # Fit clusters
    idx = 0
    condition = lambda: idx < len(np.unique(labels))
    while condition():
        label = np.unique(labels)[idx]
        mask = labels == label
        if label > 0 and mask.sum() > min_samples:
            xyz_c = hitArray[mask]
            x_c, y_c, z_c = xyz_c[:, 0], xyz_c[:, 1], xyz_c[:, 2]
            q_c = np.array(q)[mask]

            norm = np.linalg.norm(
                [
                    x_c.max() - x_c.min(),
                    y_c.max() - y_c.min(),
                    z_c.max() - z_c.min(),
                ]
            )

            # Fit the model
            inliers, outliers, score = ransacFit(xyz_c, q_c - min(q_c) + 1)

            # Refit outliers
            level_1 = np.where(mask)[0]
            level_2 = np.where(outliers)[0]
            level_3 = level_1[level_2]

            if refit_outliers and sum(outliers) > min_samples:
                outlier_labels = cluster(xyz_c[outliers])
                last_label = max(labels) + 1
                # Assign positive labels to clustered outliers and negative labels to unlclustered outliers
                for i, j in enumerate(level_3):
                    labels[j] = (outlier_labels[i] + last_label) * (
                        1 if outlier_labels[i] > 0 else -1
                    )
            else:
                # Assign negative labels to outliers
                for j in level_3:
                    labels[j] = -labels[j]

            if sum(inliers) > min_samples:
                line_fit, chi_squared = lineFit(xyz_c[inliers])
                # Degrees of freedom (number of points - number of parameters in the line fit)
                degrees_of_freedom = sum(inliers) - 1
                # Calculate the reduced chi squared
                reduced_chi_squared = chi_squared / degrees_of_freedom
                # Calculate the p-value (assuming chi-squared distribution)
                p_value = 1 - stats.chi2.cdf(chi_squared, degrees_of_freedom)

                if ax2d is not None:
                    # 2D plot
                    line_fit.plot_2d(
                        ax2d,
                        t_1=-norm / 2,
                        t_2=norm / 2,
                        c="red",
                        label=f"Track {label}",
                        zorder=10,
                    )
                if ax3d is not None:
                    # 3D plot
                    line_fit.plot_3d(
                        ax3d,
                        t_1=-norm / 2,
                        t_2=norm / 2,
                        c="red",
                        label=f"Track {label}",
                    )

                # Calculate dQ/dx
                dh = get_dh(line_fit.direction)
                dr = get_dr(np.sqrt(reduced_chi_squared))

                dq_i = dqdx(
                    xyz_c[inliers],
                    q_c[inliers],
                    line_fit,
                    dh=dh,
                    dr=dr,
                    h=norm,
                    ax=ax3d if ax3d is not None and plot_cyl else None,
                )

                q_eff = dq_i.sum() / q_c[inliers].sum()
                if dq_i.sum() != 0:
                    dq = dq_i
                else:
                    dq = 0

                metrics[label] = {
                    "Fit_line": line_fit,
                    "Fit_norm": norm,
                    "Fit_p_value": p_value,
                    "RANSAC_score": score,
                    "q_eff": q_eff,
                    "dQ": dq,
                    "dx": dh,
                }

        idx = np.unique(labels).tolist().index(label) + 1

    metrics["Fit_labels"] = labels

    return metrics

In [ ]:
def max_std(array, ax, array_max=1000, min_count_ratio=0.99, max_std_ratio=0.5):
    max_std = array.std()
    max_count = len(array)

    std = []
    count = []
    x_range = range(1, (array_max + 1), 1)
    for i in x_range:
        cut = array[(array < i)]
        std.append(cut.std())
        count.append(len(cut))

    std = np.array(std)
    count = np.array(count)
    condition = (count / max_count > min_count_ratio) & (std / max_std < max_std_ratio)
    vline = x_range[
        np.where(condition)[0][-1]
        if np.any(condition)
        else (count / max_count > min_count_ratio).argmax()
    ]

    print(
        "Max STD ratio",
        max_std_ratio,
        "limited to",
        min_count_ratio * 100,
        "% of events:",
        vline,
    )

    ax.plot(std / max_std, label="STD ratio")
    ax.plot(count / max_count, label="Event count ratio")
    ax.axvline(vline, ls="--", c="r", label=f"{min_count_ratio*100}% of events")
    ax.legend()
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_minor_locator(AutoMinorLocator())

    return vline

In [ ]:
def light_geometry(track_line, track_norm, sipm_df, light_variable="integral"):
    metrics = {"Light_distance": {}, "Light_angle": {}, f"Light_{light_variable}": {}}
    point1 = track_line.to_point(-track_norm / 2)
    point2 = track_line.to_point(track_norm / 2)
    centre = track_line.point

    iterate_df = sipm_df.dropna(subset=light_variable).copy()
    for row, sipm in iterate_df.iterrows():
        sipm_idx = (sipm["sn"], sipm["ch"])
        point3 = Point([sipm["x"], sipm["y"], 0])
        triangle = Triangle(point1, point2, point3)
        angle = triangle.angle("C")
        distance = point3.distance_point(centre)
        metrics["Light_distance"][sipm_idx] = distance
        metrics["Light_angle"][sipm_idx] = angle
        metrics[f"Light_{light_variable}"][sipm_idx] = sipm[light_variable]

    return metrics

## Plotting

In [ ]:
def create_axes(event_idx, charge, light):
    fig = plt.figure(figsize=(14, 6))
    ax3d = fig.add_subplot(121, projection="3d")
    ax2d = fig.add_subplot(122)
    fig.suptitle(
        f"Event {event_idx} - Charge = {charge} {q_unit} - Light = {light} {light_unit}"
    )
    grid_color = plt.rcParams["grid.color"]

    # Draw dead areas
    for i in range(4):
        j = 0
        if i == 3:
            j = 1
        ax2d.plot(
            np.arange(32, 64 + 1, 1) - 32 * j,
            np.arange(-80, -48 + 1, 1) + 32 * i,
            c=grid_color,
            lw=1,
        )
        ax2d.plot(
            np.arange(64, 32 - 1, -1) - 32 * j,
            np.arange(-80, -48 + 1, 1) + 32 * i,
            c=grid_color,
            lw=1,
        )

    # ax2d.plot(np.linspace(-31,0),np.linspace(8,-16), c=grid_color,lw = 1)
    ax2d.plot(np.linspace(-32, 0), np.linspace(16, -16), c=grid_color, lw=1, zorder=-1)
    ax2d.plot(np.linspace(-32, -16), np.linspace(-16, 0), c=grid_color, lw=1, zorder=-1)

    # Adjust axes
    for ax in [ax3d, ax2d]:
        ax.set_aspect("equal", adjustable="box")
        ax.set_xlim([-detector_x / 2, detector_x / 2])
        ax.set_ylim([-detector_y / 2, detector_y / 2])
        ax.set_xlabel(f"x [{xy_unit}]")
        ax.set_ylabel(f"y [{xy_unit}]")
        ax.set_xticks(np.linspace(-detector_x / 2, detector_x / 2, 5))
        ax.set_yticks(np.linspace(-detector_y / 2, detector_y / 2, 6))
        ax.grid()

    ax2d.xaxis.set_minor_locator(AutoMinorLocator(8))
    ax2d.yaxis.set_minor_locator(AutoMinorLocator(8))
    ax2d.tick_params(axis="both", which="both", right=True, top=True)

    # Adjust z-axis
    ax3d.set_zlabel(f"z [{z_unit}]")
    # ax3d.zaxis.set_major_locator(MaxNLocator(integer=True))

    return fig, (ax2d, ax3d)


def event_display(
    event_idx,
    charge_x_array,
    charge_y_array,
    charge_z_array,
    charge_time_array,
    charge_array,
    light_x_array=[],
    light_y_array=[],
    light_array=[],
    plot_cyl=False,
):
    if len(charge_x_array) < 2:
        return None

    # Plot the hits
    fig, axes = create_axes(
        event_idx, round(sum(charge_array)), round(sum(light_array))
    )
    ax2d = axes[0]
    ax3d = axes[1]

    # Group by x and y coordinates and sum the z values
    data2d = np.c_[charge_x_array, charge_y_array, charge_array]
    unique_points, indices = np.unique(data2d[:, :2], axis=0, return_inverse=True)
    q_sum = np.bincount(indices, weights=data2d[:, 2])

    # Plot the hits
    plot3d = ax3d.scatter(
        charge_x_array,
        charge_y_array,
        charge_z_array,
        c=charge_array,
        marker="s",
        s=30,
        vmin=q_sum.min(),
        vmax=q_sum.max(),
    )
    plot2d = ax2d.scatter(
        unique_points[:, 0],
        unique_points[:, 1],
        c=q_sum,
        marker="s",
        s=40,
        vmin=q_sum.min(),
        vmax=q_sum.max(),
    )
    cbar = plt.colorbar(plot2d)
    cbar.set_label(f"charge [{q_unit}]")

    # Create a design matrix
    xyz = np.c_[charge_x_array, charge_y_array, charge_z_array]

    # Cluster the hits
    labels = cluster(xyz)

    # Fit clusters
    metrics = fitClusters(xyz, charge_array, labels, ax2d, ax3d, plot_cyl)

    # Draw missing SiPMs
    grid_color = plt.rcParams["grid.color"]

    # Draw SiPMs
    sipm_plot = ax2d.scatter(
        light_x_array,
        light_y_array,
        c=light_array,
        marker="s",
        s=200,
        linewidths=1.5,
        edgecolors=grid_color,
        zorder=6,
    )

    # Draw SiPMs
    side_size = 6
    vertices_x = np.array([1, 1, -1, -1, 1]) * side_size / 2
    vertices_y = np.array([1, -1, -1, 1, 1]) * side_size / 2
    light_xy = list(zip(light_x_array, light_y_array))
    for missing_index in range(20):
        col = -48 + (missing_index % 4) * 32
        row = 64 - (missing_index // 4) * 32
        square = create_square((col, row), side_size)
        if (col, row) not in light_xy:
            ax2d.fill(col + vertices_x, vertices_y + row, c=grid_color, zorder=5)
            ax3d.add_collection3d(Poly3DCollection(square, color=grid_color))
        else:
            ax3d.add_collection3d(
                Poly3DCollection(
                    square,
                    facecolors=sipm_plot.to_rgba(
                        light_array.iloc[light_xy.index((col, row))]
                    ),
                    linewidths=0.5,
                    edgecolors=grid_color,
                )
            )

    sipm_cbar = plt.colorbar(sipm_plot)
    sipm_cbar.set_label(rf"Light {light_variable} [{light_unit}]")

    # ax2d.legend()
    # ax3d.legend()
    ax3d.set_zlim([0, ax3d.get_zlim()[1]])
    # ax3d.view_init(160, 110, -85)
    ax3d.view_init(30, 20, 100)
    # ax3d.view_init(0, 0, 0)
    # ax3d.view_init(0, 0, 90)
    fig.tight_layout()

    fig.savefig(f"event_{event_idx}.png", dpi=300)
    # plt.show()

    return metrics

In [ ]:
# Function to create a square based on center coordinates and side size
def create_square(center, side_size):
    x = [
        center[0] - side_size / 2,
        center[0] + side_size / 2,
        center[0] + side_size / 2,
        center[0] - side_size / 2,
        center[0] - side_size / 2,
    ]
    y = [
        center[1] - side_size / 2,
        center[1] - side_size / 2,
        center[1] + side_size / 2,
        center[1] + side_size / 2,
        center[1] - side_size / 2,
    ]
    z = [0, 0, 0, 0, 0]  # All z-coordinates are set to 0 to align with the xy-plane

    vertices = [(x[i], y[i], z[i]) for i in range(5)]
    square = [[vertices[0], vertices[1], vertices[2], vertices[3]]]
    return square

### Tracks

In [ ]:
# Plot dQ versus X
def plot_dQ(dQ_array, event_idx, track_idx, dh, interpolate=False):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax_twinx = ax.twinx()

    fig.suptitle(
        rf"Event {event_idx} - Track {track_idx} - $dx = {round(dh,2)}$ {dh_unit}"
    )

    mean_dQ = np.mean(dQ_array[dQ_array > 0])
    non_zero_indices = np.where(dQ_array > 0)[0]

    # Check if there are non-zero values in dQ_array
    if non_zero_indices.size > 0:
        # Find the first non-zero index and add 2 indices before it
        first_index = max(non_zero_indices[0] - 2, 0)

        # Find the last non-zero index and add 2 indices after it
        last_index = min(non_zero_indices[-1] + 2, len(dQ_array))

        new_dQ_array = dQ_array.copy()[first_index:last_index]

        if interpolate:
            new_dQ_array[1:-1] = np.where(
                new_dQ_array[1:-1] == 0,
                mean_dQ,
                new_dQ_array[1:-1],
            )

        dQ_array = new_dQ_array

    ax.axhline(
        mean_dQ / dh,
        ls="--",
        c="red",
        label=rf"Mean = ${round(mean_dQ/dh,2)}$ {q_unit} {dh_unit}$^{{-1}}$",
        lw=1,
    )
    x_range = np.arange(0, len(dQ_array) * dh, dh)[: len(dQ_array)]

    ax.step(x_range, dQ_array / dh, where="mid")
    ax.set_xlabel(rf"$x$ [{dh_unit}]")
    ax.set_ylabel(rf"$dQ/dx$ [{q_unit} {dh_unit}$^{{-1}}$]")

    ax_twinx.step(x_range, np.cumsum(dQ_array), color="C1", where="mid")
    ax_twinx.set_ylabel(f"Q [{q_unit}]")

    for axes in [ax, ax_twinx]:
        axes.xaxis.set_minor_locator(AutoMinorLocator())
        axes.yaxis.set_minor_locator(AutoMinorLocator())
        axes.xaxis.set_major_locator(MaxNLocator(integer=True))
        axes.yaxis.set_major_locator(MaxNLocator(integer=True))
        axes.tick_params(axis="both", direction="inout", which="major", top=True)

    h1, l1 = ax.get_legend_handles_labels()
    ax_twinx.legend(h1, l1, loc="lower center")

    ax.legend(loc="lower center")

    fig.tight_layout()
    fig.savefig(f"dQ_{event_idx}_{track_idx}_{round(dh,2)}.png", dpi=300)

In [ ]:
def plot_track_stats(
    metrics,
    limit_xrange=True,
    p_value_limit=0.05,
    empty_ratio_lims=(0, 1),
    min_entries=2,
    lognorm=True,
    bins=[40, 40],
):
    track_mean_dQdx = []
    track_std_dQdx = []
    track_length = []
    track_p_value = []
    dQdx_list = []

    empty_count = 0
    short_count = 0
    for entry in metrics.values():
        for track, values in entry.items():
            if isinstance(track, str) or track <= 0:
                continue

            dQ = values["dQ"]
            dx = values["dx"]
            non_zero_mask = np.where(dQ > 0)[0]

            if len(dQ[non_zero_mask]) < min_entries:
                short_count += 1
                continue

            empty_ratio = sum(dQ[non_zero_mask[0] : non_zero_mask[-1] + 1] == 0) / (
                non_zero_mask[-1] - non_zero_mask[0] + 1
            )

            if empty_ratio > empty_ratio_lims[1] or empty_ratio < empty_ratio_lims[0]:
                empty_count += 1
                continue

            dQdx = dQ[non_zero_mask[0] : non_zero_mask[-1] + 1] / dx
            x_range = np.arange(0, len(dQdx) * dx, dx)[: len(dQdx)]
            dQdx_list.append(pd.Series(dQdx, index=x_range))

            non_zero_mask = non_zero_mask[non_zero_mask[0] : non_zero_mask[-1] + 1]
            track_mean_dQdx.append(np.mean(dQdx[dQdx > 0]))
            track_std_dQdx.append(np.std(dQdx[dQdx > 0]))
            track_length.append(values["Fit_norm"])
            track_p_value.append(values["Fit_p_value"])

    print(f"Tracks with dead area outside {empty_ratio_lims} interval: {empty_count}")
    print(f"Tracks with less than {min_entries} entries: {short_count}")

    print(len(track_mean_dQdx))
    track_mean_dQdx = pd.Series(track_mean_dQdx)
    track_std_dQdx = pd.Series(track_std_dQdx)
    track_length = pd.Series(track_length)
    track_p_value = pd.Series(track_p_value)
    mask = track_mean_dQdx.notna() * track_length.notna() * track_p_value.notna()

    print(f"Remaining tracks: {sum(mask)}")

    track_mean_dQdx = track_mean_dQdx[mask]
    track_std_dQdx = track_std_dQdx[mask]
    track_length = track_length[mask]
    track_p_value = track_p_value[mask]
    track_cv_dQdx = track_std_dQdx / track_mean_dQdx

    p_mask = track_p_value <= p_value_limit
    p_bool = (1 - p_mask).sum() > 0

    print(f"Tracks with p_value > {p_value_limit}: {sum(mask)-sum(p_mask)}")

    print(f"Remaining_tracks: {sum(p_mask)}")

    # 1D histograms
    fig1 = plt.figure(figsize=(14, 6))

    ax11 = fig1.add_subplot(121)
    ax12 = fig1.add_subplot(122)

    n_all11, bins_all11, patches_all11 = ax11.hist(
        track_mean_dQdx, bins=bins[0], label="All tracks"
    )

    n_all12, bins_all12, patches_all12 = ax12.hist(
        track_length, bins=bins[0], label="All tracks"
    )

    if p_bool:
        n11, edges11, patches11 = ax11.hist(
            track_mean_dQdx[p_mask],
            bins=bins_all11,
            label=rf"p_value $\leq {p_value_limit}$",
        )
        ax12.hist(
            track_length[p_mask],
            bins=bins_all12,
            label=rf"p_value $\leq {p_value_limit}$",
        )

    ax11.set_xlabel(rf"$dQ/dx$ [{q_unit} {dh_unit}$^{{-1}}$]")
    ax11.set_title(f"{len(track_mean_dQdx)} tracks")
    # last_index = (n_all11 < 2).argmax()
    # print((n_all11 < 2))
    # print((n_all11 < 2).argmax())
    # ax11.set_xlim(-2, edges_all11[last_index])

    ax12.set_title(f"{len(track_length)} tracks")

    # 2D histograms
    fig2 = plt.figure(figsize=(14, 6))
    ax21 = fig2.add_subplot(121)

    hist2d21 = ax21.hist2d(
        track_length,
        track_mean_dQdx,
        bins=bins,
        cmin=1,
        norm=LogNorm() if lognorm else None,
    )
    ax21.set_ylabel(rf"Mean $dQ/dx$ [{q_unit} {dh_unit}$^{{-1}}$]")
    ax21.set_title("Mean dQ/dx vs. Track length")

    fit21 = np.polyfit(np.log(track_length), track_mean_dQdx, 1)

    p21 = np.poly1d(fit21)
    x2 = np.arange(min(track_length), max(track_length), 1)
    ax21.plot(x2, p21(np.log(x2)), c="salmon", ls="-", label="Log fit")

    ax22 = fig2.add_subplot(122)

    hist2d22 = ax22.hist2d(
        track_length,
        track_cv_dQdx,
        bins=bins,
        cmin=1,
        norm=LogNorm() if lognorm else None,
    )
    ax22.set_ylabel(rf"$dQ/dx$ CV")
    ax22.set_title("dQ/dx CV vs. Track length")

    fit22 = np.polyfit(np.log(track_length), track_cv_dQdx, 1)

    p22 = np.poly1d(fit22)
    ax22.plot(x2, p22(np.log(x2)), c="salmon", ls="-", label="Log fit")

    fig2.suptitle(f"{len(track_mean_dQdx)} tracks")

    axes = [ax11, ax12, ax21, ax22]
    figs = [fig1, fig2]

    fig4 = plt.figure(figsize=(7, 6))
    figs.append(fig4)

    ax4 = fig4.add_subplot(111)
    axes.append(ax4)

    hist2d4 = ax4.hist2d(
        track_length,
        track_p_value,
        bins=[bins[0], 40],
        cmin=1,
        norm=LogNorm() if lognorm else None,
    )  # , cmin=1, bins = 15)
    ax4.set_ylabel(f"Fit p_value")
    ax4.set_title("Fit p_value vs. Track length")

    cut_dQdx_series = pd.concat(
        [series for i, series in enumerate(dQdx_list) if p_mask[i]]
    )
    cut_dQdx_series = cut_dQdx_series[cut_dQdx_series > 0].dropna().sort_index()
    dQdx_series = pd.concat(dQdx_list)
    dQdx_series = dQdx_series[dQdx_series > 0].dropna().sort_index()

    fig5 = plt.figure(figsize=(7 + 7 * p_bool, 6))
    figs.append(fig5)
    ax51 = fig5.add_subplot(111 + 10 * p_bool)
    axes.append(ax51)

    ax51.hist2d(
        dQdx_series.index,
        dQdx_series,
        bins=bins,
        cmin=1,
        norm=LogNorm() if lognorm else None,
    )
    fit51 = np.polyfit(dQdx_series.index, dQdx_series, 1)
    p51 = np.poly1d(fit51)
    ax51.plot(
        dQdx_series.index,
        p51(dQdx_series.index),
        c="salmon",
        ls="-",
        label=f"Linear fit",
    )
    ax51.set_ylabel(rf"$dQ/dx$ [{q_unit} {dh_unit}$^{{-1}}$]")
    ax51.set_xlabel(rf"Residual range [{dh_unit}]")
    ax51.set_title(rf"{len(track_mean_dQdx)} tracks")

    if p_bool:
        # 2D histograms after p_value cut
        fig3 = plt.figure(figsize=(14, 6))
        figs.append(fig3)

        ax31 = fig3.add_subplot(121)
        ax32 = fig3.add_subplot(122)
        axes.extend([ax31, ax32])

        hist2d31 = ax31.hist2d(
            track_length[p_mask],
            track_mean_dQdx[p_mask],
            bins=bins,
            cmin=1,
            norm=LogNorm() if lognorm else None,
        )  # , cmin=1, bins = 15)
        ax31.set_ylabel(rf"Mean $dQ/dx$ [{q_unit} {dh_unit}$^{{-1}}$]")

        fit31 = np.polyfit(np.log(track_length[p_mask]), track_mean_dQdx[p_mask], 1)
        p31 = np.poly1d(fit31)
        x3 = np.arange(min(track_length[p_mask]), max(track_length[p_mask]), 1)
        ax31.plot(x3, p31(np.log(x3)), c="salmon", ls="-", label="Log fit")
        ax31.set_title(rf"Mean dQ/dx vs. Track length")

        hist2d32 = ax32.hist2d(
            track_length[p_mask],
            track_cv_dQdx[p_mask],
            bins=bins,
            cmin=1,
            norm=LogNorm() if lognorm else None,
        )  # , cmin=1, bins = 15)
        ax32.set_ylabel(rf"$dQ/dx$ CV")

        fit32 = np.polyfit(np.log(track_length[p_mask]), track_cv_dQdx[p_mask], 1)
        p32 = np.poly1d(fit32)
        ax32.plot(x3, p32(np.log(x3)), c="salmon", ls="-", label="Log fit")
        ax32.set_title(rf"dQ/dx CV vs. Track length")

        fig3.suptitle(
            rf"Fit p_value $\leq {p_value_limit}$ ({round(sum(p_mask)/len(p_mask)*100)}% of tracks)"
        )

        ax52 = fig5.add_subplot(122)
        axes.append(ax52)

        ax52.hist2d(
            cut_dQdx_series.index,
            cut_dQdx_series,
            bins=bins,
            cmin=1,
            norm=LogNorm() if lognorm else None,
        )
        fit52 = np.polyfit(cut_dQdx_series.index, cut_dQdx_series, 1)
        p52 = np.poly1d(fit52)
        ax52.plot(
            cut_dQdx_series.index,
            p52(cut_dQdx_series.index),
            c="salmon",
            ls="-",
            label=f"Linear fit",
        )
        ax52.set_ylabel(rf"$dQ/dx$ [{q_unit} {dh_unit}$^{{-1}}$]")
        ax52.set_xlabel(rf"Residual range [{dh_unit}]")
        ax52.set_title(
            rf"Fit p_value $\leq {p_value_limit}$ ({round(sum(p_mask)/len(p_mask)*100)}% of tracks)"
        )

    max_track_legth = np.sqrt(detector_x**2 + detector_y**2 + detector_z**2)
    max_track_legth_xy = np.sqrt(detector_x**2 + detector_y**2)
    print("Max possible track length", round(max_track_legth, 2), "mm")
    print("Max possible track lengt on xy plane", round(max_track_legth_xy, 2), "mm")
    print("Max possible vertical track length", detector_y, "mm")

    for ax in axes:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_major_locator(MaxNLocator(integer=not (ax == ax4)))
        ax.tick_params(
            axis="both", direction="inout", which="major", right=True, top=True
        )
        if ax == ax11 or ax == ax12:
            ax.set_ylabel("Counts")
        if ax != ax11:
            if not (ax == ax51 or (p_bool and ax == ax52)):
                ax.set_xlabel(f"Track length [{dh_unit}]")
            if max(track_length) > detector_y:
                ax.axvline(detector_y, c="g", ls="--", label="Max vertical length")
            if max(track_length) > max_track_legth_xy:
                ax.axvline(
                    max_track_legth_xy, c="orange", ls="--", label=r"Max length in $xy$"
                )
            if max(track_length) > max_track_legth:
                ax.axvline(max_track_legth, c="r", ls="--", label="Max length")

            if ax != ax12:
                if limit_xrange:
                    xlim = ax.get_xlim()
                    ax.set_xlim(xlim[0], min(max_track_legth + 10, xlim[1]))
                cbar = plt.colorbar(ax.collections[0])
                cbar.set_label("Counts [Log]")
        if not (not p_bool and ax == ax11):
            ax.legend(loc="upper right")

    for fig in figs:
        fig.tight_layout()

    entries = len(track_mean_dQdx)

    fig1.savefig(f"track_stats_1D_hist_{file_label}_{entries}.png", dpi=300)
    fig2.savefig(f"track_stats_2D_hist_{file_label}_{entries}.png", dpi=300)
    fig4.savefig(f"track_stats_p_value_{file_label}_{entries}.png", dpi=300)
    fig5.savefig(f"track_stats_dQdx_{file_label}_{entries}.png", dpi=300)
    if p_bool:
        fig3.savefig(f"track_stats_2D_hist_cut_{file_label}_{entries}.png", dpi=300)

In [ ]:
def plot_light_geo_stats(
    metrics,
    limit_xrange=False,
    min_count_ratio=0.99,
    max_std_ratio=0.2,
    single_track=True,
    lognorm=True,
):
    sipm_distance = []
    sipm_angle = []
    sipm_light = []
    sipm_charge = []
    non_track_keys = 4

    for event_idx, metric in metrics.items():
        if single_track and len(metric.keys()) > 1 + non_track_keys:
            continue
        for track_idx, values in metric.items():
            if not isinstance(track_idx, str) and track_idx > 0:
                track_charge = sum(values["dQ"]) / values["q_eff"]
                sipm_distance.extend(values["Light_distance"].values())
                sipm_angle.extend(values["Light_angle"].values())
                sipm_light.extend(values[f"Light_{light_variable}"].values())
                sipm_charge.extend(np.full(len(values["Light_angle"]), track_charge))

    sipm_distance = np.array(sipm_distance)
    sipm_angle = np.array(sipm_angle)
    sipm_charge = np.array(sipm_charge)
    sipm_light = np.array(sipm_light)

    max_distance = np.sqrt(detector_x**2 + detector_y**2 + detector_z**2)
    print("Max possible distance to track", round(max_distance, 2), "mm")
    print("Drift distance", detector_z, "mm")

    # if limit_xrange:
    #     sipm_distance = sipm_distance[sipm_distance <= max_distance]
    #     sipm_angle = sipm_angle[sipm_distance <= max_distance]
    #     sipm_charge = sipm_charge[sipm_distance <= max_distance]
    #     sipm_light = sipm_light[sipm_distance <= max_distance]

    sipm_distance = sipm_distance[~np.isnan(sipm_light) & (sipm_light > 0)]
    sipm_angle = sipm_angle[~np.isnan(sipm_light) & (sipm_light > 0)]
    sipm_charge = sipm_charge[~np.isnan(sipm_light) & (sipm_light > 0)]
    sipm_light = sipm_light[~np.isnan(sipm_light) & (sipm_light > 0)]

    fig1 = plt.figure(figsize=(7, 6))
    ax1 = fig1.add_subplot(111)
    axes = np.array(ax1)

    vline = max_std(
        sipm_light,
        ax1,
        array_max=100,
        max_std_ratio=max_std_ratio,
        min_count_ratio=min_count_ratio,
    )
    bins = vline

    ax1.set_xlabel("Max light integral")
    ax1.set_ylabel("Normalized value")

    fig1.suptitle("Light integral distribution")

    sipm_distance = sipm_distance[(sipm_light <= vline)]
    sipm_angle = sipm_angle[(sipm_light <= vline)]
    sipm_charge = sipm_charge[(sipm_light <= vline)]
    sipm_light = sipm_light[(sipm_light <= vline)]
    sipm_angle = np.degrees(sipm_angle)

    fig2, ax2 = plt.subplots(1, 1, figsize=(7, 6))
    axes = np.append(axes, ax2)

    hist2 = ax2.hist2d(
        sipm_distance,
        sipm_angle,
        weights=abs(sipm_light),
        bins=bins,
        cmin=1,
        norm=LogNorm() if lognorm else None,
    )
    def fit_function(x, a, b):
        return 1/(a * x + b)
    parameters, cov_matrix = curve_fit(
                    fit_function,
                    sipm_distance,
                    sipm_angle,
                )
    print(latexify.get_latex(fit_function))
    print("Parameters:")
    print(
        "\n".join(
            [
                f"{name}: {value}"
                for name, value in zip(
                    ["a", "b"], parameters
                )
            ]
        )
    )
    x2 = np.arange(min(sipm_distance), max(sipm_distance), 1)
    ax2.plot(x2,fit_function(x2, *parameters), c="salmon", ls="-", label=rf"$1/x$ fit")
    cbar2 = plt.colorbar(hist2[3])
    cbar2.set_label(rf"Light {light_variable} [{light_unit} - log]")

    ax2.set_ylabel(f"SiPM opening angle to track centre [deg]")

    fig2.suptitle(f"SiPM level light distribution - {len(sipm_light)} entries")

    fig3, axes3 = plt.subplots(1, 2, figsize=(14, 6))
    axes = np.append(axes, axes3)

    hist30 = axes3[0].hist2d(
        sipm_distance,
        sipm_light,
        bins=bins,
        cmin=1,
        norm=LogNorm() if lognorm else None,
    )
    axes3[0].set_ylabel(f"Light_{light_variable} [{light_unit}]")

    cbar30 = plt.colorbar(hist30[3])
    cbar30.set_label(rf"Counts [Log]")

    hist31 = axes3[1].hist2d(
        sipm_angle, sipm_light, bins=bins, cmin=1, norm=LogNorm() if lognorm else None
    )
    axes3[1].set_xlabel(f"SiPM opening angle to track [deg]")
    axes3[1].set_ylabel(f"Light {light_variable} [{light_unit}]")
    cbar31 = plt.colorbar(hist31[3])
    cbar31.set_label(rf"Counts [Log]")

    fig3.suptitle(f"SiPM level light distribution - {len(sipm_light)} entries")

    for ax in axes:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        if not ax == ax1:
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))

        ax.tick_params(axis="both", direction="inout", which="major", top=True)
        if ax == ax2 or ax == axes3[0]:
            if limit_xrange:
                xlim = ax.get_xlim()
                ax.set_xlim(xlim[0], min(max_distance + 10, xlim[1]))
            if max(sipm_distance) > detector_z:
                ax.axvline(detector_z, c="orange", ls="--", label="Drift distance")
            if max(sipm_distance) > max_distance:
                ax.axvline(max_distance, c="r", ls="--", label="Max distance")

            ax.set_xlabel(f"Distance from track centre [{dh_unit}]")

            ax.legend()

    for fig in [fig1, fig2, fig3]:
        fig.tight_layout()

    entries = len(sipm_light)
    fig1.savefig(f"light_geo_optimization_{file_label}_{entries}.png", dpi=300)
    fig2.savefig(f"light_geo_2D_hist_{file_label}_{entries}.png", dpi=300)
    fig3.savefig(f"light_geo_1D_hist_{file_label}_{entries}.png", dpi=300)

### Light

In [ ]:
def light_vs_charge(
    metrics,
    min_count_ratio=0.99,
    max_std_ratio=0.5,
    clusters=None,
    bin_density=1,
    log=(True, False),
):
    if isinstance(log, bool):
        log = [log, log]

    light_array = []
    charge_array = []
    for event, metric in metrics.items():
        light_array.append(metric["Total_light"])
        charge_array.append(metric["Total_charge"])

    light_array = np.array(light_array)
    charge_array = np.array(charge_array)

    mask = (charge_array > 0) & (light_array > 0)
    charge_array = charge_array[mask]
    light_array = light_array[mask]

    fig1 = plt.figure(figsize=(8, 6))
    ax1 = fig1.add_subplot(111)
    ax1.set_xlabel("Max light integral")
    ax1.set_ylabel("Normalized value")
    fig1.suptitle("Light integral distribution")
    vline = max_std(
        light_array,
        ax1,
        array_max=1000,
        min_count_ratio=min_count_ratio,
        max_std_ratio=max_std_ratio,
    )
    bins = int(vline / 20)

    charge_array = charge_array[(light_array <= vline)]
    light_array = light_array[(light_array <= vline)]

    def hist2d(x, y, ax, bins, log):
        if log:
            log_bins_x = np.exp(np.linspace(np.log(min(x) - 1), np.log(max(x)), bins))
            log_bins_y = np.exp(np.linspace(np.log(min(y)), np.log(max(y)), bins))
            bins = [log_bins_x, log_bins_y]
            ax.set_xscale("log")
            ax.set_yscale("log")

        hist = ax.hist2d(x, y, bins=bins, cmin=1)

        ax.set_xlabel(f"Event total charge [{q_unit}{' - Log' if log else ''}]")
        ax.set_ylabel(f"Light {light_variable} [{light_unit}{' - Log' if log else ''}]")
        ax.tick_params(axis="both", direction="inout", which="major", top=True)

        cbar = plt.colorbar(hist[3])
        cbar.set_label(rf"Counts")
        if not log:
            ax.xaxis.set_minor_locator(AutoMinorLocator())
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.yaxis.set_minor_locator(AutoMinorLocator())
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    def hist1d(array, ax, bin_density, log):
        # fit peak with curve_fit
        # @latexify.function(use_math_symbols=True)
        def fit_function(x, a, mu, sigma, b, c):
            return (
                # gaussian_part
                a
                * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
                / (sigma * np.sqrt(2 * np.pi))
                # exponential_part
                + b * np.exp(-c * (x - mu))
            )

        upper_bound = np.percentile(array, 95)
        array = array[array < upper_bound]
        if log:
            ax.set_xscale("log")
            bins = np.exp(
                np.linspace(
                    np.log(min(array) - 1), np.log(max(array)), int(50 * bin_density)
                )
            )
        else:
            bins = np.linspace(0, upper_bound, int(50 * bin_density))

        n, edges, patches = ax.hist(
            array, bins=bins, fill=False, ec="C0", histtype="bar"
        )

        peak_y = n.max()
        peak_x = edges[n.argmax() : n.argmax() + 1].mean()
        mean = array.mean()
        median = np.median(array)

        if not log:
            ax.xaxis.set_minor_locator(AutoMinorLocator())
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.yaxis.set_minor_locator(AutoMinorLocator())
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))

            bin_centers = 0.5 * (edges[1:] + edges[:-1])
            bin_centers = bin_centers[n > 0]
            bin_peaks = n[n > 0]
            try:
                parameters, cov_matrix = curve_fit(
                    fit_function,
                    bin_centers / min(bin_centers),
                    bin_peaks,
                    p0=[0.1, peak_x / min(bin_centers), 0.1, 0, 0.1],
                )
                x_plot = np.linspace(edges[0], edges[-1], len(bins * 2))
                y_plot = fit_function(x_plot / min(bin_centers), *parameters)

                print(latexify.get_latex(fit_function))
                print("Parameters:")
                print(
                    "\n".join(
                        [
                            f"{name}: {value}"
                            for name, value in zip(
                                ["a", "mu", "sigma", "b", "c"], parameters
                            )
                        ]
                    )
                )
                ax.plot(
                    x_plot,
                    y_plot,
                    "m",
                    label=rf"Fit ($\mu={parameters[1]*min(bin_centers):.2f}$)",
                )
            except:
                print("Fit failed")

        ax.axvline(peak_x, c="r", ls="--", label=f"Peak: {peak_x:.2f}")
        ax.axvline(median, c="orange", ls="--", label=f"Median: {median:.2f}")
        ax.axvline(mean, c="g", ls="--", label=f"Mean: {mean:.2f}")

        ax.set_ylabel(f"Counts")
        # ax.set_xlim(min(array) - 2, edges3[(edges3 < upper_bound).argmin()])
        ax.tick_params(axis="both", direction="inout", which="major", top=True)
        ax.legend()

    fig2 = plt.figure(figsize=(8, 6))
    ax2 = plt.subplot(111)
    hist2d(charge_array, light_array, ax2, bins, log[0])
    fig2.suptitle(f"Event level Light vs. Charge - {len(charge_array)} events")

    fig3 = plt.figure(figsize=(8, 6))
    ax3 = plt.subplot(111)
    ratio = charge_array / light_array
    hist1d(ratio, ax3, bin_density, log[1])
    ax3.set_xlabel(f"Event total charge / Light [{q_unit}/{light_unit}{' - Log' if log[1] else ''}]")
    fig3.suptitle(f"Event level Charge vs. Light - {len(charge_array)} events")

    fig4, axes4 = plt.subplots(1, 2, figsize=(14, 6))

    hist1d(charge_array, axes4[0], bin_density, log[1])
    axes4[0].set_xlabel(f"Event total charge [{q_unit}{' - Log' if log[1] else ''}]")
    hist1d(light_array, axes4[1], bin_density, log[1])
    axes4[1].set_xlabel(f"Event total Light [{light_unit}{' - Log' if log[1] else ''}]")

    figs = [fig1, fig2, fig3, fig4]

    if not clusters is None:
        data = pd.DataFrame(np.log(charge_array), np.log(light_array))
        kmeans = KMeans(n_clusters=clusters)
        kmeans.fit(data)
        labels = kmeans.predict(data) + 1
        populations = np.unique(labels)

        fig22, axes22 = plt.subplots(
            len(populations), 1, figsize=(8, 6 * len(populations))
        )

        fig32, axes32 = plt.subplots(
            len(populations), 1, figsize=(10, 6 * len(populations))
        )

        figs.extend([fig22, fig32])

        for idx, label in enumerate(populations):
            hist2d(
                charge_array[labels == label],
                light_array[labels == label],
                axes22[idx],
                bins,
                log[0],
            )
            axes22[idx].set_title(f"Population {label} - {sum(labels == label)} events")

            print(f"population {label}:")

            ratio = charge_array[labels == label] / light_array[labels == label]
            hist1d(ratio, axes32[idx], bin_density, log[1])
            axes32[idx].set_xlabel(f"Event total charge / Light [{q_unit}/{light_unit}{' - Log' if log[1] else ''}]")
            axes32[idx].set_title(f"Population {label} - {sum(labels == label)} events")

    for fig in figs:
        fig.tight_layout()

    events = len(ratio)
    fig1.savefig(f"light_vs_charge_optmization_{file_label}_{events}.png", dpi=300)
    fig2.savefig(f"light_vs_charge_2D_hist_{file_label}_{events}.png", dpi=300)
    fig3.savefig(f"light_vs_charge_ratio_{file_label}_{events}.png", dpi=300)

# File loading

In [ ]:
# If match dictionary already exists for this file label, load it
if not rematch_events and os.path.isfile(f"match_dict_{file_label}.json"):
    with open(f"match_dict_{file_label}.json", "r") as f:
        match_dict = json.load(f)
        match_dict = {int(key): value for key, value in match_dict.items()}

    print("Match_dict loaded from file")

In [ ]:
if not reload_file and os.path.isfile(f"charge_df_{file_label}.bz2"):
    charge_df = pd.read_csv(f"charge_df_{file_label}.bz2", index_col=0)
    if not event_list is None:
        charge_df = charge_df.loc[event_list.intersection(charge_df.index)]
    # for column in :
    charge_df[charge_df.columns[9:]] = charge_df[charge_df.columns[9:]].applymap(
        lambda x: literal_eval(x) if isinstance(x, str) else x
    )
else:
    charge_df = load_charge(charge_file, events=event_list)

In [ ]:
# Clean up charge dataframe

# Remove events with negative charge hits and without light trigger
charge_mask = (
    charge_df["event_hits_q"].apply(tuple).explode().groupby("eventID").min() > 0
) * (charge_df["trigID"].apply(len) > 0)
charge_df = charge_df[charge_mask]

print(
    f"Removed charge events: {charge_mask.count()-charge_mask.sum()}/{charge_mask.count()}"
)

In [ ]:
# If match dictionary already loaded, match loaded charge events to light events before loading light events.
light_event_list = None
if not match_dict == {}:
    light_event_list = ak.flatten(
        [
            match_dict.get(event, [])
            for event in charge_df.index
            if event_list is None or event in event_list
        ]
    )

print(f"{len(light_event_list)} light events to load.")

In [ ]:
# Load light events using light_event_list if loaded via match dictionary
if not reload_file and os.path.isfile(f"light_df_{file_label}.bz2"):
    light_df = pd.read_csv(f"light_df_{file_label}.bz2", index_col=0)
    if not light_event_list is None:
        light_df = light_df[light_df["event"].isin(light_event_list)]

else:
    light_df = load_light(
        light_file, deco="deco" in light_file, events=light_event_list
    )

In [ ]:
# Clean up light dataframe

# If match dictionary not yet loaded, create it
if match_dict == {} or rematch_events:
    match_dict = match_events(charge_df, light_df)

    # Remove light events without charge event match
    light_events = np.unique(ak.flatten(match_dict.values()))
    light_df = light_df[light_df["event"].isin(light_events)]

    print(
        f"Remaining light events with charge event match: {light_df['event'].nunique()}"
    )

# Remove charge events without associated light event
charge_df = charge_df.loc[match_dict.keys()]

print(f"Remaining charge events with light match: {len(charge_df)}")

# File saving

In [ ]:
# Only save files if all events were considered, i.e. event_list is None
if event_list is None:
    charge_df.to_csv(f"charge_df_{file_label}.bz2")
    light_df.to_csv(f"light_df_{file_label}.bz2")
    with open(f"match_dict_{file_label}.json", "w") as f:
        json.dump(match_dict, f)

# File verification

In [ ]:
charge_df.columns

In [ ]:
light_df.columns

## Histograms

In [ ]:
print("Trigger time distribution")
charge_df["trig_time"].apply(np.mean).hist()

In [ ]:
print(f"Event duration in {time_unit}")
charge_df["event_duration"].hist()

In [ ]:
print(f"Charge per hit in {q_unit}")
(charge_df["event_q"] / charge_df["event_nhits"]).hist(bins=50)

In [ ]:
print(f"Charge per hit per event in {q_unit}")
(charge_df["event_q"] / charge_df["event_nhits"]).to_frame().reset_index().plot.scatter(
    x="eventID", y=0
)

In [ ]:
print(f"Event charge in {q_unit}")
charge_df["event_q"].hist()

In [ ]:
print(f"Hits q in {q_unit}")
charge_df["event_hits_q"].apply(tuple).explode().hist()

In [ ]:
print(f"Hits z in {z_unit}")
charge_df["event_hits_z"].apply(tuple).explode().hist()

# Event display

## Fake data map

In [ ]:
def plot_fake_data(z_range):
    fake_data = generate_dead_area(z_range)
    fake_x, fake_y, fake_z = fake_data[:, 0], fake_data[:, 1], fake_data[:, 2]

    fig = plt.figure()

    ax = fig.add_subplot(111)
    ax.set_aspect("equal", adjustable="box")
    ax.scatter(fake_x, fake_y, marker="s", s=20)
    ax.set_xlim([-detector_x / 2, detector_x / 2])
    ax.set_ylim([-detector_y / 2, detector_y / 2])
    ax.set_xlabel(f"x [{xy_unit}]")
    ax.set_ylabel(f"y [{xy_unit}]")
    ax.set_xticks(np.linspace(-detector_x / 2, detector_x / 2, 5))
    ax.set_yticks(np.linspace(-detector_y / 2, detector_y / 2, 6))
    ax.xaxis.set_minor_locator(AutoMinorLocator(8))
    ax.yaxis.set_minor_locator(AutoMinorLocator(8))
    ax.grid()
    ax.tick_params(axis="both", which="both", top=True, right=True)

    fig.savefig("fake_data_map.png", dpi=300)


plot_fake_data(np.arange(-detector_z, 0, 10))

plt.show()

## Data fit

In [ ]:
# Suppress the UndefinedMetricWarning
warnings.filterwarnings("ignore", category=Warning)

In [ ]:
metrics = {}

if event_list is None:
    index_list = charge_df.index
else:
    index_list = charge_df.index.intersection(event_list)

for idx in tqdm(index_list):
    row = charge_df.loc[idx]

    charge_channelid_array = np.array(row["event_hits_channelid"])
    charge_x_array = np.array(row["event_hits_x"])
    charge_y_array = np.array(row["event_hits_y"])
    charge_z_array = np.array(row["event_hits_z"])
    charge_time_array = np.array(row["event_hits_ts"])
    charge_array = np.array(row["event_hits_q"])

    non_zero_mask = (charge_x_array != 0) * (
        charge_y_array != 0
    )  # Remove (0,0) entries
    noisy_channels_mask = np.isin(
        charge_channelid_array, channel_disable_list, invert=True
    )  # Disable channel 7
    mask = non_zero_mask * noisy_channels_mask  # Full hits mask

    # Apply boolean indexing to x, y, and z arrays
    charge_x_array = charge_x_array[mask]
    charge_y_array = charge_y_array[mask]
    charge_z_array = charge_z_array[mask]
    charge_time_array = charge_time_array[mask]
    charge_array = charge_array[mask] * charge_gain  # Convert mV to ke

    light_event = match_dict.get(idx, [idx])[0]
    light_values = light_df[light_df["event"] == light_event].dropna(
        subset=light_variable
    )
    light_x_array = light_values["x"]
    light_y_array = light_values["y"]
    light_array = light_values[light_variable]

    if len(charge_x_array) > 2:
        if idx in individual_plots:
            metrics[idx] = event_display(
                idx,
                charge_x_array,
                charge_y_array,
                charge_z_array,
                charge_time_array,
                charge_array,
                light_x_array,
                light_y_array,
                light_array,
                plot_cyl=False,
            )
            if show_figures:
                plt.show()
            else:
                plt.close()
        else:
            # Create a design matrix
            xyz = np.c_[charge_x_array, charge_y_array, charge_z_array]
            # Cluster the hits
            labels = cluster(xyz)
            # Fit clusters
            metrics[idx] = fitClusters(xyz, charge_array, labels)

        # Light metrics
        for track_idx, values in metrics[idx].items():
            if "Fit_line" not in values:
                continue
            light_metrics = light_geometry(
                track_line=values["Fit_line"],
                track_norm=values["Fit_norm"],
                sipm_df=light_values,
                light_variable=light_variable,
            )

            values["Light_distance"] = light_metrics["Light_distance"]
            values["Light_angle"] = light_metrics["Light_angle"]
            values[f"Light_{light_variable}"] = light_metrics[f"Light_{light_variable}"]

        metrics[idx][
            "Pixel_mask"
        ] = mask  # Save masks to original dataframe for reference
        metrics[idx]["Total_light"] = light_values[light_variable].sum()
        metrics[idx]["Total_charge"] = charge_array.sum()

    # else:
    #     print(f"Event {idx}: Not enough hits to fit")

In [ ]:
# Reset the warning filter (optional)
warnings.filterwarnings("default", category=Warning)

## Metrics

In [ ]:
metrics

In [ ]:
# # Save the metrics to a pickle file
# with open(f"metrics_{file_label}.pkl", "wb") as f:
#     pickle.dump(metrics, f)

# print(f"Metrics saved to metrics_{file_label}.pkl")

In [ ]:
# Load metrics from pickle file
with open(f"metrics_{file_label}.pkl", "rb") as f:
    metrics = pickle.load(f)

In [ ]:
cached_metrics = metrics
metrics = filter_metrics(
    metrics,
    p_value_limit=p_value_limit,
    min_track_length=min_track_length,
    max_tracks=1,
)

In [ ]:
metrics

# dQ/dx

In [ ]:
plot_track_stats(
    metrics,
    limit_xrange=False,
    empty_ratio_lims=(0.0, 1),
    lognorm=True,
    min_entries=2,
    p_value_limit=1,
    bins=[40, 40],
)
if show_figures:
    plt.show()
else:
    plt.close("all")

In [ ]:
for event_idx in tqdm(individual_plots, leave=False):
    if event_idx in metrics:
        for track_idx, values in metrics[event_idx].items():
            if not isinstance(track_idx, str) and track_idx > 0:
                dQ_array = values["dQ"]
                dh = values["dx"]
                plot_dQ(dQ_array, event_idx, track_idx, dh, interpolate=False)

                if show_figures:
                    plt.show()
                else:
                    plt.close("all")

# Light Geometry

In [ ]:
plot_light_geo_stats(metrics, single_track=True, limit_xrange=True, lognorm=True)
if show_figures:
    plt.show()
else:
    plt.close("all")

In [ ]:
light_vs_charge(metrics, clusters=None, bin_density=1, log=(True, False))

if show_figures:
    plt.show()
else:
    plt.close("all")

# Light wave forms

In [ ]:
if "rwf" in light_df.columns:
    for event_idx in tqdm(individual_plots, leave=True):
        values = light_df[
            (light_df["event"] == match_dict.get(event_idx, [event_idx])[0])
        ].sort_values(by=["sn", "ch"])
        for row_idx, row in values.iterrows():
            ch_idx = row["ch"]
            sn_idx = row["sn"]
            rwf = row["rwf"]

            fig = plt.figure()
            ax = plt.subplot(111)

            fig.suptitle(f"Event {event_idx} - ADC {sn_idx} - Channel {ch_idx}")
            ax.plot(rwf[:-1])
            ax.axhline(np.median(rwf), ls="--", c="red")

            ax.set_ylabel(f"Light waveform [{light_unit}]")
            ax.set_xlabel("Time [Arbitrary units]")

            fig.tight_layout()

            if show_figures:
                plt.show()
            else:
                plt.close()